In [1]:
import pandas as pd
import numpy as np

# import the required classes and definitions from Python-MIP
from mip import Model, xsum, maximize, BINARY, INTEGER

Using Python-MIP package version 1.5.1


In [2]:
#sort the top500 by element_type

top500 = pd.read_csv('top500.csv')
top500sorted = top500.sort_values('element_type')
predictions = pd.read_csv('predictions')
top500sorted = top500sorted.merge(predictions, how='left', on='id_')
top500sorted.fillna(0, inplace=True)

#lists for the dream team model, sorted by position
eltype = list(top500sorted['element_type'])
form = list(top500sorted['form'])
score = list(top500sorted['total_points'])
price = list(top500sorted['now_cost'])
team = list(top500sorted['team'])
predictions = list(top500sorted['predictions'])
form_score = list(pd.Series(form) * pd.Series(score))


In [3]:
#make a list of our main columns of interest
simple_columns = ['web_name', 'element_type', 'total_points', 'predictions', 'now_cost', 'form',
                 'goals_scored', 'assists', 'clean_sheets', 'team', 'selected_by_percent']

#make a smaller df for easy viewing of dream_team
top500simple = top500sorted[simple_columns]
top500simple.shape

(500, 11)

In [4]:
#find cutoffs for the element_types

eltype_counts = top500sorted['element_type'].value_counts().sort_index()

cutoff1 = eltype_counts[1]
cutoff2 = cutoff1 + eltype_counts[2]
cutoff3 = cutoff2 + eltype_counts[3]
cutoff4 = cutoff3 + eltype_counts[4]

print(cutoff1, cutoff2, cutoff3, cutoff4)
eltype_counts

44 217 428 500


1     44
2    173
3    211
4     72
Name: element_type, dtype: int64

In [5]:
df = top500sorted
type(df)

pandas.core.frame.DataFrame

In [6]:
df = df.reset_index()
df.head()

,index,id_,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,...,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,predictions
0,0,412,0,0,0,0.0,0.0,0,19236,0,...,0,5885,117,18676,314,0.0,0.0,Ruddy,0,0.391242
1,1,513,0,0,0,0.0,0.0,0,18656,0,...,0,4545,199,8813,199,0.0,0.0,Gomes,0,0.391242
2,2,235,0,5,308,100.0,100.0,3,51940,0,...,63,853759,3848,1036466,15697,0.5,11.7,de Gea,1,3.750277
3,3,168,0,2,392,0.0,0.0,8,17745,0,...,84,1039780,52915,437514,15639,0.6,15.6,Schmeichel,0,4.097969
4,4,237,0,0,0,0.0,0.0,0,6744,0,...,0,633,23,4483,63,0.0,0.0,Grant,0,0.000000


In [7]:
ars = list(df[df.team == 1].index)
avl = list(df[df.team == 2].index)
bou = list(df[df.team == 3].index)
bha = list(df[df.team == 4].index)
bur = list(df[df.team == 5].index)
che = list(df[df.team == 6].index)
cry = list(df[df.team == 7].index)
eve = list(df[df.team == 8].index)
lei = list(df[df.team == 9].index)
liv = list(df[df.team == 10].index)
mci = list(df[df.team == 11].index)
mun = list(df[df.team == 12].index)
new = list(df[df.team == 13].index)
nor = list(df[df.team == 14].index)
shu = list(df[df.team == 15].index)
sou = list(df[df.team == 16].index)
tot = list(df[df.team == 17].index)
wat = list(df[df.team == 18].index)
whu = list(df[df.team == 19].index)
wol = list(df[df.team == 20].index)

500

In [11]:
# The model

# define the problem data, change the profit list here
profit = score #run with current_week_points and compare
weight = price
c = 830
n = len(weight)

#create an empty maximization
m = Model('team_select')

#add the binary decision variables to model m and store their references in a list x
x = [m.add_var(var_type=BINARY) for i in range(n)]

#define the objective function of this model 
m.objective = maximize(xsum(profit[i] * x[i] for i in range(n)))

#add the capacity constraints
m += xsum(weight[i] * x[i] for i in range(n)) <= c

m += xsum(x) <= 11

m += xsum(eltype[i] * x[i] for i in range(cutoff1)) >= 1
m += xsum(eltype[i] * x[i] for i in range(cutoff1)) <= 1

m += xsum(eltype[i] * x[i] for i in range(cutoff1,cutoff2)) >= 6
m += xsum(eltype[i] * x[i] for i in range(cutoff1,cutoff2)) <= 10

m += xsum(eltype[i] * x[i] for i in range(cutoff2,cutoff3)) >= 6
m += xsum(eltype[i] * x[i] for i in range(cutoff2,cutoff3)) <= 15

m += xsum(eltype[i] * x[i] for i in range(cutoff3,cutoff4)) >= 4
m += xsum(eltype[i] * x[i] for i in range(cutoff3,cutoff4)) <= 12


# max 3 per team

m += xsum(df.team[i] * x[i] for i in ars) <= 3
m += xsum(df.team[i] * x[i] for i in avl) <= 6
m += xsum(df.team[i] * x[i] for i in bou) <= 9
m += xsum(df.team[i] * x[i] for i in bha) <= 12
m += xsum(df.team[i] * x[i] for i in bur) <= 15
m += xsum(df.team[i] * x[i] for i in che) <= 18
m += xsum(df.team[i] * x[i] for i in cry) <= 21
m += xsum(df.team[i] * x[i] for i in eve) <= 24
m += xsum(df.team[i] * x[i] for i in lei) <= 27
m += xsum(df.team[i] * x[i] for i in liv) <= 30
m += xsum(df.team[i] * x[i] for i in mci) <= 33
m += xsum(df.team[i] * x[i] for i in mun) <= 36
m += xsum(df.team[i] * x[i] for i in new) <= 39
m += xsum(df.team[i] * x[i] for i in nor) <= 42
m += xsum(df.team[i] * x[i] for i in shu) <= 45
m += xsum(df.team[i] * x[i] for i in sou) <= 48
m += xsum(df.team[i] * x[i] for i in tot) <= 51
m += xsum(df.team[i] * x[i] for i in wat) <= 54
m += xsum(df.team[i] * x[i] for i in whu) <= 57
m += xsum(df.team[i] * x[i] for i in wol) <= 60



#Optimize the model
m.optimize()

# Compute the solution, a list of the selected items
selected = [i for i in range(n) if x[i].x >= 0.99]
print('selected items: {}'.format(selected))
dream_team = top500simple.iloc[selected,:]
dream_team

selected items: [12, 152, 156, 193, 213, 399, 400, 402, 443, 444, 499]


,web_name,element_type,total_points,predictions,now_cost,form,goals_scored,assists,clean_sheets,team,selected_by_percent
12,Ryan,1,81,3.984042,48,3.0,0,0,5,4,17.2
152,Evans,2,86,4.937442,53,4.4,1,2,8,9,7.0
156,Baldock,2,91,5.077926,50,5.0,2,4,7,15,11.4
193,Lundstram,2,98,4.231807,51,3.3,3,3,7,15,46.8
213,Alexander-Arnold,2,112,3.131095,75,8.0,2,10,6,10,37.7
399,Mané,3,140,3.140761,123,7.7,11,8,7,10,39.4
400,De Bruyne,3,141,4.645587,106,7.5,7,14,8,11,51.1
402,Maddison,3,103,4.890176,77,5.1,6,5,7,9,27.1
443,Ings,4,118,5.109018,67,7.3,13,1,2,16,20.0
444,Abraham,4,115,4.933760,78,4.6,12,4,5,6,33.2


In [12]:
sum(dream_team['now_cost'])

829

In [13]:
sum(dream_team['total_points'])

1229